In [ ]:
import boto3
import random

In [ ]:
s3 = boto3.resource('s3')

### Create Buckets and Enable Replication

In [ ]:
import boto3
s3 = boto3.client('s3')

In [ ]:
west_bucket='kfs3-west-dest-dev'
east_bucket='kfs3-east-dest-dev'

In [ ]:
s3.create_bucket(
    Bucket=east_bucket
)

In [ ]:
# East versioning
s3.put_bucket_versioning(
    Bucket=east_bucket,
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

In [ ]:
# West bucket
s3.create_bucket(
    Bucket=west_bucket,
    CreateBucketConfiguration={
        'LocationConstraint':'us-west-2'
    }
)

In [ ]:
# West versioning
s3.put_bucket_versioning(
    Bucket=west_bucket,
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

In [ ]:
import os
account_no = os.environ['PRODUCER_ACCOUNT_NO']

In [ ]:
assume_role_policy_document="""{
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Principal":{
            "Service":"s3.amazonaws.com"
         },
         "Action":"sts:AssumeRole"
      }
   ]
}"""

In [ ]:
iam = boto3.client('iam')
cr = iam.create_role(
    RoleName='repl-sample-role',
    AssumeRolePolicyDocument=assume_role_policy_document
)

print(cr)

In [ ]:
repl_policy_document_west2east="""{
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Action":[
            "s3:GetObjectVersionForReplication",
            "s3:GetObjectVersionAcl",
            "s3:GetObjectVersionTagging"
         ],
         "Resource":[
            "arn:aws:s3:::""" + west_bucket + """/*"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ListBucket",
            "s3:GetReplicationConfiguration"
         ],
         "Resource":[
            "arn:aws:s3:::""" + west_bucket + """/*"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ReplicateObject",
            "s3:ReplicateDelete",
            "s3:ReplicateTags"
         ],
         "Resource":"arn:aws:s3:::""" + east_bucket + """/*"
      }
   ]
}"""


In [ ]:
repl_policy_document_east2west="""{
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Action":[
            "s3:GetObjectVersionForReplication",
            "s3:GetObjectVersionAcl",
            "s3:GetObjectVersionTagging"
         ],
         "Resource":[
            "arn:aws:s3:::""" + east_bucket + """/*"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ListBucket",
            "s3:GetReplicationConfiguration"
         ],
         "Resource":[
            "arn:aws:s3:::""" + east_bucket + """/*"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ReplicateObject",
            "s3:ReplicateDelete",
            "s3:ReplicateTags"
         ],
         "Resource":"arn:aws:s3:::""" + west_bucket + """/*"
      }
   ]
}"""


In [ ]:
cp = iam.create_policy(
    PolicyName='repl-sample-policy-e2w',
    PolicyDocument=repl_policy_document_east2west
)

print(cp)

In [ ]:
cp = iam.create_policy(
    PolicyName='repl-sample-policy-w2e',
    PolicyDocument=repl_policy_document_west2east
)

print(cp)

In [ ]:
ap = iam.attach_role_policy(
    RoleName='repl-sample-role',
    PolicyArn="arn:aws:iam::{}:policy/repl-sample-policy-e2w".format(account_no)
)
print(ap)



In [ ]:
ap = iam.attach_role_policy(
    RoleName='repl-sample-role',
    PolicyArn="arn:aws:iam::{}:policy/repl-sample-policy-w2e".format(account_no)
)
print(ap)

In [ ]:
role_arn='arn:aws:iam::{}:role/repl-sample-role'.format(account_no)
print(role_arn)

In [ ]:
# Configure replication - east to west

# Note to future self: if Prefix is not included you will receive a MalformedXML error with
# no explanation as to the root cause.
s3.put_bucket_replication(
    Bucket=east_bucket,
    ReplicationConfiguration={
        "Role":role_arn,
        "Rules":[
            {
                "ID":"rule1",
                "Status":"Enabled",
                "Destination": {
                    "Bucket" : "arn:aws:s3:::" + west_bucket
                },
                "Prefix":""
            }
        ]
    }
)

In [ ]:
# Configure replication - west to east

# Note to future self: if Prefix is not included you will receive a MalformedXML error with
# no explanation as to the root cause.
s3.put_bucket_replication(
    Bucket=west_bucket,
    ReplicationConfiguration={
        "Role":role_arn,
        "Rules":[
            {
                "ID":"shithead",
                "Status":"Enabled",
                "Destination": {
                    "Bucket" : "arn:aws:s3:::" + east_bucket
                },
                "Prefix":""
            }
        ]
    }
)

In [ ]:
## TODO - ADD CLEAN UP

In [ ]:
s3.delete_bucket_replication(
    Bucket=east_bucket
)

In [ ]:
s3.delete_bucket_replication(
    Bucket=west_bucket
)

In [ ]:
def empty_and_delete(bucket, s3client):
    response = s3client.list_object_versions(
        Bucket=bucket,
        Prefix='',
        MaxKeys=100
    )


    print (response)

    if 'Versions' in response:
        versions = response['Versions']
        for v in versions:
            print (v['Key'])
    else:
        print('No versions')


    if 'Versions' in response:
        for version in versions:
            response = s3client.delete_object(
                Bucket=bucket1,
                Key=version['Key'],
                VersionId=version['VersionId']
            )

            print(response)

    response = s3client.delete_bucket(
        Bucket=bucket
    )

    print(response)

In [ ]:
empty_and_delete(west_bucket, s3)

In [ ]:
empty_and_delete(east_bucket, s3)

In [ ]:
policy_arn_e2w="arn:aws:iam::{}:policy/repl-sample-policy-e2w".format(account_no)
policy_arn_w2e="arn:aws:iam::{}:policy/repl-sample-policy-w2e".format(account_no)

In [ ]:
iam.detach_role_policy(
    RoleName='repl-sample-role',
    PolicyArn=policy_arn_e2w
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn_e2w
)

In [ ]:
iam.detach_role_policy(
    RoleName='repl-sample-role',
    PolicyArn=policy_arn_w2e
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn_w2e
)

In [ ]:
iam.delete_role(
    RoleName='repl-sample-role'
)